In [1]:
import numpy as np
import pandas as pd
import keras 
from keras.layers import *
from keras.activations import *
from keras.models import *
import keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import itertools
%matplotlib inline

Using TensorFlow backend.


In [2]:
df = pd.read_csv("please.csv")
n_class = 7
turn_to_rgb = True

In [3]:
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = 4.
histogramNormType = 0
L2HysThreshold = 2.0000000000000001e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize,blockSize,blockStride,cellSize,nbins,derivAperture,winSigma,
                        histogramNormType,L2HysThreshold,gammaCorrection,nlevels)
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)


In [4]:
def get_file(df):
    X_train = []
    y_train = []
    X_test  = []
    y_test  = []
    HOG_train = []
    HOG_test = []
    
    for i in df.values :
#         print(i[1].split())
        pixls = np.array([float(j) for j in i[1].split()])
        w = int(np.sqrt(len(pixls)))
        h = w
        
        pixls = pixls.reshape((w, h, 1)).astype(np.uint8)
        if turn_to_rgb :
            pixls = np.array(Image.fromarray(pixls[:, :, 0]).convert("RGB"))
        
        HOG = hog.compute(pixls.astype(np.uint8),winStride,padding,locations).reshape((-1))
        y = np.eye(n_class)[int(i[0])] 
        
        if i[-1] == "Training":
            X_train.append(pixls)
            y_train.append(y)
            HOG_train.append(HOG)
        else :
            X_test.append(pixls)
            y_test.append(y)
            HOG_test.append(HOG)
            
            
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    HOG_train = np.array(HOG_train)
    HOG_test  = np.array(HOG_test)
    
    
    return X_train, y_train, X_test, y_test, HOG_train, HOG_test

In [5]:
def prepare_data(X):
    
    X = X.astype(np.float32)
    X /= 255.0
    
    
    return X

In [6]:
def get_final_model(model, shape_HOG):
    in1 = model.input
    out = model.output
    
    h_input = Input(shape_HOG)
    
    X = Flatten()(out)
    
    X = Concatenate()([X, h_input])
    X = BatchNormalization()(X)
    X = Dropout(0.45)(X)
    X = Dense(1024, activation=relu)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.4)(X)
    X = Dense(256, activation=relu)(X)
    X = BatchNormalization()(X)    
    X = Dropout(0.3)(X)
    X = Dense(64, activation=relu)(X)
    X = BatchNormalization()(X)
    X = Dropout(0.3)(X)
    X = Dense(7, activation=softmax)(X)
    
    model_final = Model([in1, h_input], X)
    
    return model_final
    
    

In [7]:
def custom_loss(yTrue,yPred) :
    yPred = 0.3/n_class + 0.7*yPred
    return K.categorical_crossentropy(yTrue, yPred)

In [8]:
df.head()

,emotion,pixels,Usage
0,0,249 255 254 252 255 239 255 162 10 45 58 7...,Test
1,0,74 77 78 73 63 57 57 59 58 58 53 5...,Training
2,0,33 22 19 53 47 39 28 46 93 104 21 1...,Training
3,0,226 219 200 189 204 214 181 133 105 93 39 3...,Training
4,0,233 242 237 242 229 227 237 196 169 163 164 17...,Training


In [9]:
train = False 
if train ==  True :
    X_train, y_train, X_test, y_test, HOG_tarin, HOG_test = get_file(df)
    X_train = prepare_data(X_train)
    X_test = prepare_data(X_test)
    model = keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(48, 48, 3), pooling=None)
    model.save("downloaded_model.h5")
    model_f = get_final_model(model, [HOG_tarin[0].shape[0]])
    model_f.compile(keras.optimizers.Adam(1e-3, decay=0.0), loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])
    model_f.fit([X_train, HOG_tarin], y_train, batch_size=32, epochs=3, validation_data=[[X_test, HOG_test], y_test])
    model_f.compile(keras.optimizers.Adam(4e-5, decay=0.0), loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])
    model_f.fit([X_train, HOG_tarin], y_train, batch_size=32, epochs=50, validation_data=[[X_test, HOG_test], y_test])
    print(model_f.evaluate([X_test, HOG_test], y_test))
    model_f.save("secondTry.h5")

In [10]:
if train ==  True :
    print(model_f.evaluate([X_test, HOG_test], y_test))
    model_f.save("secondTry.h5")

In [11]:
if train == False :
    model_f = load_model("secondTry.h5")

In [12]:
dict1 = dict(zip(itertools.count(), ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]))

In [13]:
import cv2
import sys

# cascPath = sys.argv[1]
# faceCascade = cv2.CascadeClassifier(cascPath)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

#     faces = faceCascade.detectMultiScale(
#         gray,
#         scaleFactor=1.1,
#         minNeighbors=5,
#         minSize=(30, 30),
#         flags=cv2.CASCADE_SCALE_IMAGE
#     )
    faces = face_cascade.detectMultiScale(gray, 1.3, 5, minSize=(30, 30))

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        im = Image.fromarray(gray[y:y+h,x:x+w].astype(np.uint8)).resize((48,48)).convert("RGB")
        face = np.array(im).reshape((1,48,48,3))
        HOG = hog.compute(face[0],winStride,padding,locations).reshape((1,-1))
        face = face.astype(np.float32)
        face /= 255.0
        pred = model_f.predict([face, HOG])
        cl = pred[0].argmax()
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, dict1[cl], (x,y), font, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 